In [4]:
import requests
import json
import pandas as pd    # 引用套件並縮寫為 pd
from bs4 import BeautifulSoup
# https://www.dcard.tw/f/makeup

GET 美妝版首頁，Use BeautifulSoup with html, wrtie a txt

 

In [5]:
test = open("data/test.txt","w",encoding='UTF-8')

p = requests.Session()                            # 把這一次的request都算在同一個session裡
url=requests.get("https://www.dcard.tw/f/makeup") # 請求的網址
soup = BeautifulSoup(url.text,"html.parser")
sel = soup.select("h2.tgn9uw-2 a")
a=[]
for link in sel:
    a.append(link["href"])
    print('https://www.dcard.tw'+link.get('href'))

https://www.dcard.tw/f/makeup/p/235066496
https://www.dcard.tw/f/makeup/p/235064384
https://www.dcard.tw/f/makeup/p/235063858
https://www.dcard.tw/f/makeup/p/235064997
https://www.dcard.tw/f/makeup/p/235066635
https://www.dcard.tw/f/makeup/p/235065508
https://www.dcard.tw/f/makeup/p/235064240
https://www.dcard.tw/f/makeup/p/235069295
https://www.dcard.tw/f/makeup/p/235067793
https://www.dcard.tw/f/makeup/p/235065114
https://www.dcard.tw/f/makeup/p/235065941
https://www.dcard.tw/f/makeup/p/235066790
https://www.dcard.tw/f/makeup/p/235066210
https://www.dcard.tw/f/makeup/p/235067506
https://www.dcard.tw/f/makeup/p/235067391
https://www.dcard.tw/f/makeup/p/235066878
https://www.dcard.tw/f/makeup/p/235066383
https://www.dcard.tw/f/makeup/p/235070137
https://www.dcard.tw/f/makeup/p/235069313
https://www.dcard.tw/f/makeup/p/235066564
https://www.dcard.tw/f/makeup/p/235066351
https://www.dcard.tw/f/makeup/p/235070394
https://www.dcard.tw/f/makeup/p/235069880
https://www.dcard.tw/f/makeup/p/23

In [6]:
#a
url = "https://www.dcard.tw"+ a[1]
url

'https://www.dcard.tw/f/makeup/p/235064384'

根據把首頁往下滑，發現網站會對server送一個GET請求下一個30篇文章
- 2018(G) https://ithelp.ithome.com.tw/articles/10203216
- 2019(G) https://tlyu0419.github.io/2019/04/06/Crawl-Dcard/
- 2020 https://blog.jiatool.com/posts/dcard_api_v2/

### API root

- 看板資訊 http://dcard.tw/_api/forums
- 文章資訊 http://dcard.tw/_api/forums/{看板名稱}/posts
- 文章內文	http://dcard.tw/_api/posts/{文章編號}
- 文章內引用連結	http://dcard.tw/_api/posts/{文章編號}/links
- 文章內留言	http://dcard.tw/_api/posts/{文章編號}/comments

### API wirte
- 文章數量 https://www.dcard.tw/_api/posts?limit=100
- 文章分頁 http://dcard.tw/_api/posts/?before=文章id (before 回傳由新到舊 after舊到新)
- 文章留言 http://dcard.tw/_api/posts/文章id/comments
- 留言分頁 http://dcard.tw/_api/posts/文章id/comments?after=floor數字
- 熱門文章 https://dcard.tw/_api/posts?popular=true

### others
文章資訊(meta) 與 文章內留言 預設使用熱門度 (popularity) 作為排序，而且一次請求 (request) 中只回應 30 筆。
* 想要讓這兩項使用 時間 排序，可在 GET 參數中加入 popular=false
* 欲取得更多的 文章資訊(meta)，可以使用 before={某文章編號}來獲得早於 #{某文章編號} 的另外 30 筆 資訊。
* 欲取得更多的 文章內留言，可以使用 after={某樓層} 來獲得大於 #{某樓層} 的另外 30 筆 留言。

In [7]:
# 固定碼測試
def Catch_test(ID):
    url = 'https://www.dcard.tw/_api/posts/'+str(ID)
    requ = requests.get(url)
    rejs = requ.json()
    #rejs
    return(pd.DataFrame(
        data=
        [{'ID':rejs['id'],
          'title':rejs['title'],
          'content':rejs['content'],
          'excerpt':rejs['excerpt'],
          'createdAt':rejs['createdAt'],
          'updatedAt': rejs['updatedAt' ],
          'forumName':rejs['forumName'],
          'forumAlias':rejs['forumAlias'],
          'gender':rejs['gender'],
          'topics':rejs['topics']}],
        columns=['ID', 'title', 'content', 'excerpt', 'createdAt', 
                 'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
          )
Catch_test(235020702)

,ID,title,content,excerpt,createdAt,updatedAt,forumName,forumAlias,gender,topics
0,235020702,發現冬天好適合用粉霜喔!,入冬後我臉已開始乾TT\n之前夏天用MUF粉餅 遮瑕力跟控油力都不錯\n但冬天用真的不夠保濕...,入冬後我臉已開始乾TT，之前夏天用MUF粉餅 遮瑕力跟控油力都不錯，但冬天用真的不夠保濕，最...,2020-12-21T02:26:52.040Z,2020-12-21T02:34:47.300Z,美妝,makeup,F,NaN


In [8]:
url = 'https://www.dcard.tw/_api/posts/235020702'  #測試1
#url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true'  #測試2
requ = requests.get(url)
rejs = requ.json()
rejs

{'id': 235020702,
 'title': '發現冬天好適合用粉霜喔!',
 'content': '入冬後我臉已開始乾TT\n之前夏天用MUF粉餅 遮瑕力跟控油力都不錯\n但冬天用真的不夠保濕\n最近發現了有粉霜這種材質 之前只知道粉底液很保濕 \n但粉底液我常常沒用完就過期了XD\n就納入了新的粉霜~covermark的水鑽石\nhttps://i.imgur.com/o2D1n1A.jpg\n單看上去會覺得很像膏類製質地\n但抹開來就化掉了 輕拍在臉上保濕力超好\n以前我眼下遮瑕都會另外遮\n但這個直接可以把我黑眼圈遮掉 很不錯用\n分享我今年的冬天蜜糖給大家><',
 'excerpt': '入冬後我臉已開始乾TT，之前夏天用MUF粉餅 遮瑕力跟控油力都不錯，但冬天用真的不夠保濕，最近發現了有粉霜這種材質 之前只知道粉底液很保濕，但粉底液我常常沒用完就過期了XD，就納入了新的粉霜~cove',
 'anonymousSchool': False,
 'anonymousDepartment': True,
 'pinned': False,
 'forumId': 'cbd5285f-3cba-4bfc-86d0-1ab52d201459',
 'replyId': None,
 'createdAt': '2020-12-21T02:26:52.040Z',
 'updatedAt': '2020-12-21T02:34:47.300Z',
 'commentCount': 31,
 'likeCount': 704,
 'tags': [],
 'topics': ['粉霜'],
 'supportedReactions': None,
 'withNickname': False,
 'reportReason': '',
 'hiddenByAuthor': False,
 'meta': {'layout': 'classic'},
 'forumName': '美妝',
 'forumAlias': 'makeup',
 'nsfw': False,
 'school': '文藻外語大學',
 'replyTitle': None,
 'gender': 'F',
 'personaSubscriptable': True,
 're

In [9]:
# slope the title
def Catch(ID):
    url = 'https://www.dcard.tw/_api/posts/'+str(ID)
    requ = requests.get(url)
    rejs = requ.json()
    #rejs
    return(pd.DataFrame(
        data=
        [{'ID':rejs['id'],
          'title':rejs['title'],
          'content':rejs['content'],
          'excerpt':rejs['excerpt'],
          'createdAt':rejs['createdAt'],
          'updatedAt': rejs['updatedAt' ],
          'forumName':rejs['forumName'],
          'forumAlias':rejs['forumAlias'],
          'gender':rejs['gender'],
          'topics':rejs['topics']}],
        columns=['ID', 'title', 'content', 'excerpt', 'createdAt', 
                 'updatedAt', 'forumName', 'forumAlias','gender', 'topics '])
          )

In [16]:
# 熱門美妝版測試 (100筆)
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100'
resq = requests.get(url)
rejs = resq.json()
df = pd.DataFrame()
for i in range(len(rejs)):
    df = df.append(Catch(rejs[i]['id']),ignore_index=True)
    if i % 2 == 0:
        print('data reading...',i)
print(df.shape)
df.to_csv('data/Op_Dcard_markeup.csv', index=False)
df
# 99	235031315	#請益 #問 LA girl 遮瑕挑色

data reading... 0
data reading... 2
data reading... 4
data reading... 6
data reading... 8
data reading... 10
data reading... 12
data reading... 14
data reading... 16
data reading... 18
data reading... 20
data reading... 22
data reading... 24
data reading... 26
data reading... 28
data reading... 30
data reading... 32
data reading... 34
data reading... 36
data reading... 38
data reading... 40
data reading... 42
data reading... 44
data reading... 46
data reading... 48
data reading... 50
data reading... 52
data reading... 54
data reading... 56
data reading... 58
data reading... 60
data reading... 62
data reading... 64
data reading... 66
data reading... 68
data reading... 70
(72, 10)


,ID,title,content,excerpt,createdAt,updatedAt,forumName,forumAlias,gender,topics
0,235066496,歲末美妝板十大熱門文章回顧，與你一起細細回味ヾ(●゜▽゜●)♡,本週就是跨年夜了！\n👿惡靈惡靈終於要畫下句點了～\n今天小天使來帶大家回顧一下 2020 ...,本週就是跨年夜了！，惡靈惡靈終於要畫下句點了～今天小天使來帶大家回顧一下 2020 美妝板上...,2020-12-28T08:12:22.551Z,2020-12-28T09:23:23.273Z,美妝,makeup,D,NaN
1,235064384,#分享 Cezanne 塞尚的持久潤澤唇膏出了超顯白的「中藥色」！,Cezanne \n持久潤澤唇膏\n401\n-\n🦖塞尚之前很火的持久潤澤唇膏出了2個新色...,Cezanne，持久潤澤唇膏，401，-，🦖塞尚之前很火的持久潤澤唇膏出了2個新色！分別是珊...,2020-12-28T01:24:54.780Z,2020-12-28T01:24:54.780Z,美妝,makeup,M,NaN
2,235063858,#分享 #分享 仙氣乾燥玫瑰眼妝🌹教學分享,https://i.imgur.com/rShoqsD.jpg\n來了之前被敲碗的教學😝\n...,來了之前被敲碗的教學，/閃亮的教學奉上🤲，是不是很簡單！！，-，好可惜尾牙沒有抽到獎，連最可...,2020-12-27T19:36:18.499Z,2020-12-27T19:36:18.499Z,美妝,makeup,F,NaN
3,235064997,這盒眉粉白色大家都怎麼用😂,https://i.imgur.com/H8mUnxX.jpg\nIntegrate這盒用一...,Integrate這盒用一年了 還是這麼多 就第二個比較淺的咖啡色看見鐵片而已，後面兩個都凹...,2020-12-28T03:54:00.898Z,2020-12-28T13:06:45.915Z,美妝,makeup,F,NaN
4,235066635,☃️新品7支Solone大藝術家玩色刷具☃️,午安\n之前有買過Solone的hello kitty聯名的鼻影刷\n覺得刷毛很蓬鬆 而且很...,午安，之前有買過Solone的hello kitty聯名的鼻影刷，覺得刷毛很蓬鬆 而且很好暈...,2020-12-28T08:34:19.551Z,2020-12-28T08:39:05.056Z,美妝,makeup,F,NaN
...,...,...,...,...,...,...,...,...,...,...
67,235068370,#請益 想請問大家正假貨 #霓淨思乳液,想請問一下有用過霓淨思乳液的各位\n剛剛發現產品外觀很明顯不一樣！！\n一個HA藍底白字、另...,想請問一下有用過霓淨思乳液的各位，剛剛發現產品外觀很明顯不一樣！！，一個HA藍底白字、另一個...,2020-12-28T13:22:26.664Z,2020-12-28T15:20:24.696Z,美妝,makeup,F,NaN
68,235071953,徵臉部按摩MD,小妹最近去進修美容課程\n需要徵臉部按摩的模特\n\n卸妝、洗臉、去角質最後臉部按摩\n時間...,小妹最近去進修美容課程，需要徵臉部按摩的模特，卸妝、洗臉、去角質最後臉部按摩，時間約為30～...,2020-12-29T04:24:46.094Z,2020-12-29T04:24:46.094Z,美妝,makeup,F,NaN
69,235063708,#討論 請問有人知道這牌子評價如何嗎～,在instagram看到的廣告，想問各位水水的意見🙏\n\nhttps://i.imgur....,在instagram看到的廣告，想問各位水水的意見,2020-12-27T18:52:44.910Z,2020-12-27T18:52:44.910Z,美妝,makeup,F,NaN
70,235066407,#討論 求推薦粉底液！,我是混合偏乾ㄉ女子~日前剛把super stay空瓶\n想要換新的粉底液！！\n希望各位大大推薦😳,我是混合偏乾ㄉ女子~日前剛把super stay空瓶，想要換新的粉底液！！，希望各位大大推薦,2020-12-28T07:54:30.415Z,2020-12-28T07:54:30.415Z,美妝,makeup,F,NaN


# ---------------正式版本分隔線---------------
- 測試版本代號：EGG_1.2 
- 開發人員： EGG Jerry Three


In [11]:
last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID '235031315'
url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
resq = requests.get(url)
rejs = resq.json()
for i in range(len(rejs)):
    df = df.append(Catch(rejs[i]['id']), ignore_index=True)
print(df.shape)


df

(72, 10)


,ID,title,content,excerpt,createdAt,updatedAt,forumName,forumAlias,gender,topics
0,235066496,歲末美妝板十大熱門文章回顧，與你一起細細回味ヾ(●゜▽゜●)♡,本週就是跨年夜了！\n👿惡靈惡靈終於要畫下句點了～\n今天小天使來帶大家回顧一下 2020 ...,本週就是跨年夜了！，惡靈惡靈終於要畫下句點了～今天小天使來帶大家回顧一下 2020 美妝板上...,2020-12-28T08:12:22.551Z,2020-12-28T09:23:23.273Z,美妝,makeup,D,NaN
1,235064384,#分享 Cezanne 塞尚的持久潤澤唇膏出了超顯白的「中藥色」！,Cezanne \n持久潤澤唇膏\n401\n-\n🦖塞尚之前很火的持久潤澤唇膏出了2個新色...,Cezanne，持久潤澤唇膏，401，-，🦖塞尚之前很火的持久潤澤唇膏出了2個新色！分別是珊...,2020-12-28T01:24:54.780Z,2020-12-28T01:24:54.780Z,美妝,makeup,M,NaN
2,235063858,#分享 #分享 仙氣乾燥玫瑰眼妝🌹教學分享,https://i.imgur.com/rShoqsD.jpg\n來了之前被敲碗的教學😝\n...,來了之前被敲碗的教學，/閃亮的教學奉上🤲，是不是很簡單！！，-，好可惜尾牙沒有抽到獎，連最可...,2020-12-27T19:36:18.499Z,2020-12-27T19:36:18.499Z,美妝,makeup,F,NaN
3,235064997,這盒眉粉白色大家都怎麼用😂,https://i.imgur.com/H8mUnxX.jpg\nIntegrate這盒用一...,Integrate這盒用一年了 還是這麼多 就第二個比較淺的咖啡色看見鐵片而已，後面兩個都凹...,2020-12-28T03:54:00.898Z,2020-12-28T13:06:45.915Z,美妝,makeup,F,NaN
4,235066635,☃️新品7支Solone大藝術家玩色刷具☃️,午安\n之前有買過Solone的hello kitty聯名的鼻影刷\n覺得刷毛很蓬鬆 而且很...,午安，之前有買過Solone的hello kitty聯名的鼻影刷，覺得刷毛很蓬鬆 而且很好暈...,2020-12-28T08:34:19.551Z,2020-12-28T08:39:05.056Z,美妝,makeup,F,NaN
...,...,...,...,...,...,...,...,...,...,...
67,235068370,#請益 想請問大家正假貨 #霓淨思乳液,想請問一下有用過霓淨思乳液的各位\n剛剛發現產品外觀很明顯不一樣！！\n一個HA藍底白字、另...,想請問一下有用過霓淨思乳液的各位，剛剛發現產品外觀很明顯不一樣！！，一個HA藍底白字、另一個...,2020-12-28T13:22:26.664Z,2020-12-28T15:20:24.696Z,美妝,makeup,F,NaN
68,235071953,徵臉部按摩MD,小妹最近去進修美容課程\n需要徵臉部按摩的模特\n\n卸妝、洗臉、去角質最後臉部按摩\n時間...,小妹最近去進修美容課程，需要徵臉部按摩的模特，卸妝、洗臉、去角質最後臉部按摩，時間約為30～...,2020-12-29T04:24:46.094Z,2020-12-29T04:24:46.094Z,美妝,makeup,F,NaN
69,235063708,#討論 請問有人知道這牌子評價如何嗎～,在instagram看到的廣告，想問各位水水的意見🙏\n\nhttps://i.imgur....,在instagram看到的廣告，想問各位水水的意見,2020-12-27T18:52:44.910Z,2020-12-27T18:52:44.910Z,美妝,makeup,F,NaN
70,235066407,#討論 求推薦粉底液！,我是混合偏乾ㄉ女子~日前剛把super stay空瓶\n想要換新的粉底液！！\n希望各位大大推薦😳,我是混合偏乾ㄉ女子~日前剛把super stay空瓶，想要換新的粉底液！！，希望各位大大推薦,2020-12-28T07:54:30.415Z,2020-12-28T07:54:30.415Z,美妝,makeup,F,NaN


In [12]:

#df

In [13]:
##ok ^
for j in range(3):
    last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
print(df.shape)
#df

(72, 10)


In [14]:
# 透過迴圈讀取10*100篇文章，若需讀取更多資料，可以將range(10)中的數值提升
for j in range(10):
    last = str(int(df.tail(1).ID)) # 找出爬出資料的最後一筆ID
    url = 'http://dcard.tw/_api/forums/makeup/posts?popular=true&limit=100&before=' + last
    resq = requests.get(url)
    rejs = resq.json()
    for i in range(len(rejs)):
        df = df.append(Catch(rejs[i]['id']), ignore_index=True)
        if i % 20 == 0:
            print('data reading...',i)
print(df.shape)
df


(72, 10)


,ID,title,content,excerpt,createdAt,updatedAt,forumName,forumAlias,gender,topics
0,235066496,歲末美妝板十大熱門文章回顧，與你一起細細回味ヾ(●゜▽゜●)♡,本週就是跨年夜了！\n👿惡靈惡靈終於要畫下句點了～\n今天小天使來帶大家回顧一下 2020 ...,本週就是跨年夜了！，惡靈惡靈終於要畫下句點了～今天小天使來帶大家回顧一下 2020 美妝板上...,2020-12-28T08:12:22.551Z,2020-12-28T09:23:23.273Z,美妝,makeup,D,NaN
1,235064384,#分享 Cezanne 塞尚的持久潤澤唇膏出了超顯白的「中藥色」！,Cezanne \n持久潤澤唇膏\n401\n-\n🦖塞尚之前很火的持久潤澤唇膏出了2個新色...,Cezanne，持久潤澤唇膏，401，-，🦖塞尚之前很火的持久潤澤唇膏出了2個新色！分別是珊...,2020-12-28T01:24:54.780Z,2020-12-28T01:24:54.780Z,美妝,makeup,M,NaN
2,235063858,#分享 #分享 仙氣乾燥玫瑰眼妝🌹教學分享,https://i.imgur.com/rShoqsD.jpg\n來了之前被敲碗的教學😝\n...,來了之前被敲碗的教學，/閃亮的教學奉上🤲，是不是很簡單！！，-，好可惜尾牙沒有抽到獎，連最可...,2020-12-27T19:36:18.499Z,2020-12-27T19:36:18.499Z,美妝,makeup,F,NaN
3,235064997,這盒眉粉白色大家都怎麼用😂,https://i.imgur.com/H8mUnxX.jpg\nIntegrate這盒用一...,Integrate這盒用一年了 還是這麼多 就第二個比較淺的咖啡色看見鐵片而已，後面兩個都凹...,2020-12-28T03:54:00.898Z,2020-12-28T13:06:45.915Z,美妝,makeup,F,NaN
4,235066635,☃️新品7支Solone大藝術家玩色刷具☃️,午安\n之前有買過Solone的hello kitty聯名的鼻影刷\n覺得刷毛很蓬鬆 而且很...,午安，之前有買過Solone的hello kitty聯名的鼻影刷，覺得刷毛很蓬鬆 而且很好暈...,2020-12-28T08:34:19.551Z,2020-12-28T08:39:05.056Z,美妝,makeup,F,NaN
...,...,...,...,...,...,...,...,...,...,...
67,235068370,#請益 想請問大家正假貨 #霓淨思乳液,想請問一下有用過霓淨思乳液的各位\n剛剛發現產品外觀很明顯不一樣！！\n一個HA藍底白字、另...,想請問一下有用過霓淨思乳液的各位，剛剛發現產品外觀很明顯不一樣！！，一個HA藍底白字、另一個...,2020-12-28T13:22:26.664Z,2020-12-28T15:20:24.696Z,美妝,makeup,F,NaN
68,235071953,徵臉部按摩MD,小妹最近去進修美容課程\n需要徵臉部按摩的模特\n\n卸妝、洗臉、去角質最後臉部按摩\n時間...,小妹最近去進修美容課程，需要徵臉部按摩的模特，卸妝、洗臉、去角質最後臉部按摩，時間約為30～...,2020-12-29T04:24:46.094Z,2020-12-29T04:24:46.094Z,美妝,makeup,F,NaN
69,235063708,#討論 請問有人知道這牌子評價如何嗎～,在instagram看到的廣告，想問各位水水的意見🙏\n\nhttps://i.imgur....,在instagram看到的廣告，想問各位水水的意見,2020-12-27T18:52:44.910Z,2020-12-27T18:52:44.910Z,美妝,makeup,F,NaN
70,235066407,#討論 求推薦粉底液！,我是混合偏乾ㄉ女子~日前剛把super stay空瓶\n想要換新的粉底液！！\n希望各位大大推薦😳,我是混合偏乾ㄉ女子~日前剛把super stay空瓶，想要換新的粉底液！！，希望各位大大推薦,2020-12-28T07:54:30.415Z,2020-12-28T07:54:30.415Z,美妝,makeup,F,NaN


In [15]:
df.to_excel('data/Dcard_markeup.excel', index=False)

ValueError: No engine for filetype: 'excel'

欄位	說明	備註
ID	編號	
title	標題	
content	內容	
excerpt	摘要	
createdAt	發布時間	
updatedAt	更新時間	
commentCount	留言數	
forumName	分類	中文
forumAlias	分類	英文
gender	性別	
likeCount	心情數量	
reactions	心情細項	把以上心情細分為「愛心」、「哈哈」、「跪」、「森77」、「驚訝」等類型
topics	標籤	